In [1]:
]activate ..; instantiate

  Updating registry at `~/.julia/registries/General`


┌ Warning: Some registries failed to update:
│     — `~/.julia/registries/General` — registry dirty
└ @ Pkg.API /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:157


In [56]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using CuArrays
imgs = MNIST.images()
X = hcat(float.(reshape.(imgs, :))...) |> gpu
labels = MNIST.labels()
Y = onehotbatch(labels, 0:9) |> gpu
dataset = repeated((X, Y), 200)
loss(x, y) = crossentropy(m(x), y)
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))
evalcb = () -> @show(loss(X, Y))
summary.((X,Y))

("784×60000 CuArray{Float32,2}", "10×60000 Flux.OneHotMatrix{CuArray{Flux.OneHotVector,1}}")

In [64]:
m = Chain(Dense(28^2, 32, relu),Dense(32, 10),softmax) |> gpu
opt = ADAM(params(m))
@time Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))
@show accuracy(X, Y)
# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu
@show accuracy(tX, tY)

loss(X, Y) = 2.34511f0 (tracked)
  2.893798 seconds (520.41 k allocations: 19.438 MiB, 35.21% gc time)
accuracy(X, Y) = 0.92175
accuracy(tX, tY) = 0.9232


0.9232

In [58]:
using Knet: Knet, KnetArray, param, param0, nll
kX = KnetArray(Array(X))
kY = labels
kdata = repeated((kX,kY),200)
Knet.gpu()

0

In [59]:
struct kChain; layers; kChain(ls::Tuple)=new(ls); end
kChain(ls...)=kChain(ls)
(c::kChain)(x) = (for l in c.layers; x = l(x); end; x)
struct kDense; w; b; f; end
kDense(nx::Int,ny::Int,fn=identity)=kDense(param(ny,nx),param0(ny),fn)
(d::kDense)(x) = d.f.(d.w * x .+ d.b)
iters(n)=(J->((n-=1)>0))

iters (generic function with 1 method)

In [62]:
# much faster but converges to .82 instead of .92 ???
km = kChain(kDense(28^2, 32, Knet.relu),kDense(32, 10))
@show Knet.accuracy(km,kX,kY)
@time Knet.train!(km,kdata,callback=iters(length(kdata)))
@show Knet.accuracy(km,kX,kY)

Knet.accuracy(km, kX, kY) = 0.09818333333333333
  0.558124 seconds (292.63 k allocations: 236.360 MiB, 11.85% gc time)
Knet.accuracy(km, kX, kY) = 0.8211833333333334


0.8211833333333334

In [63]:
Knet.gc()
Knet.gpuinfo()

(:dev, 0, :total, 16945512448, :free, 14639366144, :used, 2306146304)
(:bytes, 426114724, :size, 213057362, :alloc, 2, :avail, 0)
(:bytes, 728217, :size, 104031, :alloc, 7, :avail, 0)
(:bytes, 9030, :size, 1290, :alloc, 7, :avail, 0)
(:bytes, 896, :size, 128, :alloc, 7, :avail, 0)
(:bytes, 280, :size, 40, :alloc, 7, :avail, 0)
(:bytes, 0, :size, 8388608, :alloc, 0, :avail, 0)
(:bytes, 0, :size, 2642245, :alloc, 0, :avail, 0)
(:bytes, 0, :size, 262144, :alloc, 0, :avail, 0)
(:totbytes, 426853147, :avail, 0, :ptrs, 30)
(:membytes, 426853147, :avail, 0, :limit, 1227210404, :pools, 8)
